In [1]:
import requests
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import time

## Fruits and vegetables price info

In [46]:
def get_price_info_df(start_year, end_year, crop = None):
    """
    Return price info dataframe from start_year to end_year.
    Max limit of data response : 9999.
    Min limit of start date : 101.01.01
    Query string : 
        StartDate : 101.01.01
        EndDate : 101.01.01
        Market : 三重
        Crop= 椰子
    """
    url = 'https://data.coa.gov.tw/Service/OpenData/FromM/FarmTransData.aspx?{}&{}'
    if crop:
        url = 'https://data.coa.gov.tw/Service/OpenData/FromM/FarmTransData.aspx?{}&{}&Crop=' + crop
    df = pd.DataFrame()
    for year in range (start_year, end_year + 1):
        for month in range (1, 13):
            month = '{:02d}'.format(month)
            for day in range (1, 32):
                day = '{:02d}'.format(day)

                startDate = 'StartDate={}.{}.{}'.format(year, month, day)
                endDate = 'EndDate={}.{}.{}'.format(year, month, day)
                try:
                    r = requests.get(url.format(startDate, endDate))
                    if r.status_code == requests.codes.ok :
                        obj = json.loads(r.text)
                        df = pd.concat([df, pd.DataFrame(data = obj)], ignore_index = True)
                except Exception as e:
                    print('date : {}/{}/{} error.'.format(year, month, day), e)
        print('year : {} completed.'.format(year))
        
    df.to_csv('{}_{}_{}_target_price_info.csv'.format(crop, start_year, end_year))
    return df

## Typhoon 

In [40]:
"""
1958 ~ 2020 typhoon info
"""
url = 'https://rdc28.cwb.gov.tw/TDB/public/warning_typhoon_list/'
driver = webdriver.Chrome(executable_path=r'/Users/chenboyu/Desktop/NCTU_Big_Data_Final_Project/utils/chromedriver')
driver.get(url)

time.sleep(3)

pageSource = driver.page_source
html = BeautifulSoup(pageSource, 'html.parser')
typhoon_all = html.find_all('tr')

info_title = ['年份', '颱風編號', '颱風名稱', 'Column 5', '侵臺路徑分類','警報期間', 'Column 8', '近臺強度', 
              '近臺最低氣壓(hPa)', '近臺最大風速(m/s)', '近臺7級風暴風半徑(km)', '近臺10級風暴風半徑(km)', '警報發布報數']

typhoon_df = pd.DataFrame(columns = info_title)

for typhoon in typhoon_all[1:-1]:
    info = []
    for typhoon_info in typhoon:
        if index != 0 and len(typhoon_info.text) != 0:
            info.append(typhoon_info.text)
    typhoon_df = typhoon_df.append(dict(zip(info_title, info)), ignore_index = True)

## Price return

In [257]:
# df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10], ignore_index = True)
# df = df.sort_values(['作物代號', '市場名稱', '交易日期'], ignore_index = True)
df_typhoon = pd.read_csv('../data/101_108_typhoon_with_date.csv')
df_price = pd.read_csv('../data/101_108_target_price_info.csv')

In [535]:
# df_price.insert(8, "1_day_return", None)
# df_price.insert(9, "3_day_return", None)
# df_price.insert(10, "5_day_return", None)
for n in np.unique(df_price['作物代號']):
    for k in np.unique(df_price['市場代號']):
        series = df_price[(df_price['作物代號'] == n) & (df_price['市場代號'] == k)]
        if len(series) > 0:
            df_price.insert(13, '{}_{}_1_day_return'.format(n, k), series['平均價'].pct_change(1))
            df_price.insert(14, '{}_{}_3_day_return'.format(n, k), series['平均價'].pct_change(3))
            df_price.insert(15, '{}_{}_5_day_return'.format(n, k), series['平均價'].pct_change(5))
            df_price['1_day_return'].update(df_price.pop('{}_{}_1_day_return'.format(n, k)))
            df_price['3_day_return'].update(df_price.pop('{}_{}_3_day_return'.format(n, k)))
            df_price['5_day_return'].update(df_price.pop('{}_{}_5_day_return'.format(n, k)))

## Concat with typhoon

In [537]:
for i in range(len(df_typhoon)):
    typhoon_series = (df_price['DateTime'] >= df_typhoon['Start'][i]) & (df_price['DateTime'] <= df_typhoon['End'][i])
    for j in typhoon_series[typhoon_series].index:
        df_price['入台時間'][j] = df_typhoon['Start'][i]
        df_price['離台時間'][j] = df_typhoon['End'][i]
        df_price['強度'][j] = df_typhoon['近臺強度'][i]
        df_price['警報發布報數'][j] = df_typhoon['警報發布報數'][i]

In [554]:
df_price.to_csv('../data/101_108_price_with_typhoon.csv', index = False)
df_price.to_csv('../data/101_108_price_with_typhoon_utf8.csv', index = False, encoding='utf-8')